In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_5_for_train.txt"
testDataPath = "s3://corpus-2/dataset/corpus_5_new.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        test_data = [[] for i in range(num)]
        
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])
                
        with open(testDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                test_data[index].append([content,label])

        for i in range(num):
            # np.random.shuffle(train_data[i])
            train_data[i], test_data[i] = shuffle(train_data[i], test_data[i])
            train_data[i] = np.asarray(train_data[i])
            test_data[i] = np.asarray(test_data[i])

        train_data, test_data = shuffle(train_data, test_data)
        return train_data, test_data
    
    
    train_data = []
    test_data = []
    
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
            
    with open(testDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            test_data.append([content,label])
            
    train_data, test_data = shuffle(train_data, test_data)
    return np.asarray(train_data), np.asarray(test_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(train_data, test_data, split_type):
    
    print(split_type)
    
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)
        
        te_x = []
        te_y = []
        for ti in test_data:
            x_test, y_test = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            te_x.append(x_test)
            te_y.append(y_test)

        te_x = np.asarray(te_x)
        te_y = np.asarray(te_y)
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(te_x[test_i],te_y[test_i])
            test_acc.append(classifier.train(
                X_train=train_x,
                y_train=train_y,
                X_eval=test_x,
                y_eval=test_y,
                categories=categories,
                epochs=50
            ))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        te_x, te_y = process_file(test_data[:,0], test_data[:,1], word_to_id, num_classes, seq_length)
        # print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            test_acc.append(classifier.train(
                X_train=tx[train_i],
                y_train=ty[train_i],
                X_eval=te_x[test_i],
                y_eval=te_y[test_i],
                categories=categories,
                epochs=50
            ))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
train_data, test_data = dataset_split(split_info["expert"])

In [5]:
train_data[3][:3]

array([[list(['in', '6450', '-0708', 'what', 'was', 'the', 'change', 'of', 'the', 'number', 'of', 'frontcourt', 'obtained', 'a', 'total', 'of', '048', 'points', '?']),
        10],
       [list(['is', 'there', 'any', 'clustering', 'of', 'weights', 'evident', 'based', 'off', 'of', 'this', 'group', 'of', 'director', '?']),
        9],
       [list(['what', 'continent', 'has', 'the', 'health', 'with', 'the', 'lowest', 'failures', '?']),
        4]], dtype=object)

In [6]:
test_data[3][:3]

array([[list(['in', '2010', '-2019', 'what', 'was', 'the', 'change', 'of', 'the', 'number', 'of', 'students', 'obtained', 'a', 'total', 'of', '270', 'points', '?']),
        10],
       [list(['is', 'there', 'any', 'clustering', 'of', 'weights', 'evident', 'based', 'off', 'of', 'this', 'group', 'of', 'students', '?']),
        9],
       [list(['what', 'continent', 'has', 'the', 'country', 'with', 'the', 'lowest', 'income', '?']),
        4]], dtype=object)

In [7]:
def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)  # label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [8]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)
 
seq_length = 41  # seq_length = 37  TREC

Loading GloVe!
Completed!
400000 100 400000


In [9]:
class Classifier:

    def __init__(self, model, input_length, output_length):
        self.model = model
        self.input_length = input_length
        self.output_length = output_length

    def compile(self, batch_size=32):
        self._ds_x = tf.placeholder(tf.float32, [None, self.input_length])
        self._ds_y = tf.placeholder(tf.float32, [None, self.output_length])

        ds = tf.data.Dataset.from_tensor_slices((self._ds_x, self._ds_y))
        ds = ds.batch(batch_size)

        self._ds_it = ds.make_initializable_iterator()
        self._input, self._labels = self._ds_it.get_next()

        self._features = self.model(self._input)
        self._output = _create_dense_layer(self._features, self.output_length)

        self._create_acc_computations()
        self._create_backpropagation()

    def _create_acc_computations(self):
        self._predictions = tf.argmax(self._output, 1)
        labels = tf.argmax(self._labels, 1)
        self._accuracy = tf.reduce_mean(
            tf.cast(tf.equal(self._predictions, labels), 'float32'))

    def _create_backpropagation(self):
        losses = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self._output,
            labels=self._labels)
        self._loss = tf.reduce_mean(losses)

        optimizer = tf.train.AdamOptimizer(0.001)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        grads_and_vars = optimizer.compute_gradients(self._loss)

        self._train_op = optimizer.apply_gradients(
            grads_and_vars, global_step=global_step)

    def summary(self):
        print('input:', self._input.shape)
        self.model.summary()
        print('output:', self._output.shape)

    def train(self, X_train, y_train, X_eval, y_eval, categories, epochs=20, require_improve=3):
        
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())
        
        best_vac_acc = 0.0
        last_improved = 0
        
        for e in range(epochs):
            start_time = time.time()
            loss, acc = self._train(X_train, y_train, session)
            duration = time.time() - start_time

            val_loss, val_acc = self._eval(X_eval, y_eval, session)
            
            if val_acc > best_vac_acc:
                best_vac_acc = val_acc
                last_improved = e
                improved_str = '*'
            else:
                improved_str = ''
            
            output = 'Epoch: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
            print(output.format(e + 1, loss, acc, val_loss, val_acc, duration, improved_str))
            
            if e - last_improved > require_improve:
                print("No optimization for a long time, auto-stopping...")
                
                y_test_cls = np.argmax(y_eval, 1)  # 获得类别
                y_test_pred_cls = np.argmax(self.predict(X_eval, session), 1)
                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                
                # evaluate
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                print("Confusion Matrix...")
                print(metrics.confusion_matrix(y_test_cls, y_test_pred_cls))
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                break
        # endfor
        session.close()
        return accuracy_score

    def _train(self, X_train, y_train, session):
        import numpy as np

        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_train,
                self._ds_y: y_train
            })
        loss, acc, = [], []
        while True:
            try:
                _, vloss, vacc = session.run(
                    fetches=[self._train_op, self._loss, self._accuracy])

                loss.append(vloss)
                acc.append(vacc)
            except tf.errors.OutOfRangeError:
                break
        # endwhile

        loss, acc = np.mean(loss), np.mean(acc)
        return loss, acc

    def _eval(self, X_val, y_val, session):
        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_val,
                self._ds_y: y_val
            })

        loss, acc, = 0, 0
        while True:
            try:
                l, vloss, vacc = session.run(
                    fetches=[self._labels, self._loss, self._accuracy])

                loss += vloss * len(l)
                acc += vacc * len(l)
            except tf.errors.OutOfRangeError:
                break

        return loss / len(X_val), acc / len(X_val)

    def predict(self, X, session):
        session.run(self._ds_it.initializer,
                         feed_dict={
                             self._ds_x: X,
                             self._ds_y: np.empty((len(X), self.output_length))
                         }
                         )

        pred = list()
        while True:
            try:
                ppred = session.run(tf.nn.softmax(self._output))

                pred.extend(map(lambda l: l.tolist(), ppred))
            except tf.errors.OutOfRangeError:
                break

        return pred

def _create_dense_layer(x, output_length):
    '''Creates a dense layer
    '''
    input_size = x.shape[1].value
    W = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[input_size, output_length],
            stddev=0.1))
    b = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[output_length]))

    dense = tf.nn.xw_plus_b(x, W, b)

    return dense

In [10]:
class ZhouCLSTMModel:
    '''
    Implementation proposal of: https://arxiv.org/pdf/1511.08630
    '''
    def __init__(self, embedding,
        conv_size    = 3,
        conv_filters = 300,
        drop_rate    = 0.5,
        lstm_units   = 150):
        '''Constructor.
        # Parameters:
        conv_size: Size of the convolutions. Number of words that takes each
            convolution step.
        conv_filters: Number of convolution filters.
        drop_rate: Drop rate for the final output of the LSTM layer.
        lstm_units: Size of the states of the LSTM layer.
        '''
        self._embedding    = embedding
        self._conv_size    = conv_size
        self._conv_filters = conv_filters
        self._drop_rate    = drop_rate
        self._lstm_units   = lstm_units

    def __call__(self, input):
        self._embedding_tf = self._create_embedding_layer(
            self._embedding, input)
        
        self._convolution_tf = self._create_convolutional_layers(
            self._conv_size,
            self._conv_filters,
            self._drop_rate,
            self._embedding_tf)
        
        self._lstm_tf = self._create_lstm_layer(
            self._lstm_units,
            self._convolution_tf)
        
        return self._lstm_tf

    def summary(self):
        print('embedding:', str(self._embedding_tf.shape))
        print('conv:', str(self._convolution_tf.shape))
        print('lstm:', str(self._lstm_tf.shape))

    def _create_embedding_layer(self, embedding, input_x):
        embedding = tf.Variable(initial_value=embedding)

        embedded_chars = tf.nn.embedding_lookup(
            embedding, tf.cast(input_x, 'int32'))

        return embedded_chars

    def _create_convolutional_layers(self,
        conv_size, num_filters, drop_rate, embedding):
        
        filter_height = conv_size
        filter_width = embedding.shape[2].value

        filter_shape = [filter_height, filter_width, 1, num_filters]

        W = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=filter_shape,
                stddev=0.1))
        b = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=[num_filters]))

        embedding_expanded = tf.expand_dims(embedding, -1)
        conv = tf.nn.conv2d(
            input=embedding_expanded,
            filter=W,
            strides=[1,1,1,1],
            padding='VALID')
        conv_reduced = tf.reshape(
            tensor=conv,
            shape=[-1, conv.shape[1], conv.shape[3]])

        bias = tf.nn.bias_add(conv_reduced, b)
        c = tf.nn.relu(bias)

        d = tf.nn.dropout(c, keep_prob=drop_rate)
        return d

    def _create_lstm_layer(self, lstm_units, conv_input):
        lstm_cell = tf.nn.rnn_cell.LSTMCell(lstm_units)
        sequence = tf.unstack(conv_input, axis=1)
        (_, (h, _)) = tf.nn.static_rnn(lstm_cell, sequence, dtype=tf.float32)

        return h

In [11]:
word_vector = embedding.astype('float32')
model = ZhouCLSTMModel(embedding=word_vector)

classifier = Classifier(
    model=model,
    input_length=seq_length,
    output_length=num_classes)

classifier.compile(batch_size=32)
classifier.summary()

input: (?, 41)
embedding: (?, 41, 100)
conv: (?, 39, 300)
lstm: (?, 150)
output: (?, 10)


In [12]:
kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data, test_data = dataset_split(info)
    test_acc_split.append(train_split_data(train_data, test_data, split_type))

random
Fold:  1
Epoch: 1, Train Loss:  1.551, Train Acc: 47.23%, Val Loss:  0.897, Val Acc: 71.15%, Time: 9.88s *
Epoch: 2, Train Loss: 0.6935, Train Acc: 77.94%, Val Loss: 0.6169, Val Acc: 81.20%, Time: 7.20s *
Epoch: 3, Train Loss: 0.4193, Train Acc: 86.28%, Val Loss: 0.4788, Val Acc: 85.40%, Time: 7.34s *
Epoch: 4, Train Loss: 0.2904, Train Acc: 90.57%, Val Loss: 0.4179, Val Acc: 87.11%, Time: 7.15s *
Epoch: 5, Train Loss: 0.2035, Train Acc: 93.45%, Val Loss: 0.4517, Val Acc: 86.89%, Time: 7.34s 
Epoch: 6, Train Loss: 0.1477, Train Acc: 95.27%, Val Loss: 0.4254, Val Acc: 88.75%, Time: 7.28s *
Epoch: 7, Train Loss: 0.1116, Train Acc: 96.33%, Val Loss: 0.4478, Val Acc: 88.96%, Time: 7.26s *
Epoch: 8, Train Loss: 0.08987, Train Acc: 97.10%, Val Loss: 0.4605, Val Acc: 89.53%, Time: 7.15s *
Epoch: 9, Train Loss: 0.07614, Train Acc: 97.42%, Val Loss: 0.4823, Val Acc: 88.46%, Time: 7.10s 
Epoch: 10, Train Loss:  0.065, Train Acc: 97.74%, Val Loss: 0.4638, Val Acc: 88.89%, Time: 7.21s 
Epoc

Epoch: 1, Train Loss:  1.495, Train Acc: 49.06%, Val Loss:  0.882, Val Acc: 73.58%, Time: 8.94s *
Epoch: 2, Train Loss:  0.665, Train Acc: 78.74%, Val Loss: 0.5957, Val Acc: 82.62%, Time: 7.15s *
Epoch: 3, Train Loss:  0.409, Train Acc: 86.84%, Val Loss: 0.5159, Val Acc: 84.33%, Time: 7.23s *
Epoch: 4, Train Loss: 0.2729, Train Acc: 91.26%, Val Loss: 0.5144, Val Acc: 84.97%, Time: 7.11s *
Epoch: 5, Train Loss: 0.2013, Train Acc: 93.59%, Val Loss:  0.477, Val Acc: 86.97%, Time: 7.10s *
Epoch: 6, Train Loss: 0.1495, Train Acc: 95.19%, Val Loss:  0.526, Val Acc: 86.18%, Time: 7.18s 
Epoch: 7, Train Loss:  0.113, Train Acc: 96.03%, Val Loss: 0.5333, Val Acc: 86.97%, Time: 7.14s 
Epoch: 8, Train Loss: 0.08782, Train Acc: 97.00%, Val Loss: 0.4977, Val Acc: 87.82%, Time: 7.16s *
Epoch: 9, Train Loss: 0.07694, Train Acc: 97.43%, Val Loss: 0.5857, Val Acc: 86.61%, Time: 7.23s 
Epoch: 10, Train Loss: 0.06841, Train Acc: 97.74%, Val Loss: 0.5531, Val Acc: 87.54%, Time: 7.11s 
Epoch: 11, Train Los

Epoch: 1, Train Loss:  1.541, Train Acc: 47.35%, Val Loss: 0.9078, Val Acc: 69.78%, Time: 9.18s *
Epoch: 2, Train Loss: 0.6907, Train Acc: 77.95%, Val Loss: 0.5409, Val Acc: 82.47%, Time: 7.13s *
Epoch: 3, Train Loss: 0.4193, Train Acc: 86.97%, Val Loss: 0.4849, Val Acc: 85.03%, Time: 7.11s *
Epoch: 4, Train Loss: 0.2797, Train Acc: 90.98%, Val Loss: 0.4363, Val Acc: 86.39%, Time: 7.11s *
Epoch: 5, Train Loss: 0.2002, Train Acc: 93.45%, Val Loss: 0.4102, Val Acc: 87.24%, Time: 7.18s *
Epoch: 6, Train Loss: 0.1453, Train Acc: 95.28%, Val Loss:  0.439, Val Acc: 88.24%, Time: 7.23s *
Epoch: 7, Train Loss: 0.1178, Train Acc: 95.99%, Val Loss: 0.3659, Val Acc: 89.95%, Time: 7.27s *
Epoch: 8, Train Loss: 0.08681, Train Acc: 97.06%, Val Loss: 0.4547, Val Acc: 88.31%, Time: 7.25s 
Epoch: 9, Train Loss: 0.08226, Train Acc: 97.17%, Val Loss: 0.4439, Val Acc: 89.31%, Time: 7.33s 
Epoch: 10, Train Loss: 0.06442, Train Acc: 97.64%, Val Loss: 0.4395, Val Acc: 89.52%, Time: 7.28s 
Epoch: 11, Train Lo

Epoch: 1, Train Loss:  1.588, Train Acc: 45.45%, Val Loss: 0.9697, Val Acc: 69.00%, Time: 9.03s *
Epoch: 2, Train Loss: 0.7109, Train Acc: 76.84%, Val Loss: 0.6446, Val Acc: 79.76%, Time: 7.21s *
Epoch: 3, Train Loss: 0.4355, Train Acc: 86.04%, Val Loss: 0.4798, Val Acc: 85.46%, Time: 7.21s *
Epoch: 4, Train Loss:  0.284, Train Acc: 90.86%, Val Loss: 0.5155, Val Acc: 85.39%, Time: 7.13s 
Epoch: 5, Train Loss: 0.2108, Train Acc: 93.13%, Val Loss: 0.4987, Val Acc: 86.03%, Time: 7.34s *
Epoch: 6, Train Loss: 0.1526, Train Acc: 95.08%, Val Loss: 0.4651, Val Acc: 87.74%, Time: 7.39s *
Epoch: 7, Train Loss:  0.118, Train Acc: 96.16%, Val Loss: 0.5493, Val Acc: 86.03%, Time: 7.22s 
Epoch: 8, Train Loss: 0.09022, Train Acc: 96.98%, Val Loss: 0.5243, Val Acc: 87.24%, Time: 7.26s 
Epoch: 9, Train Loss: 0.08035, Train Acc: 97.39%, Val Loss: 0.5468, Val Acc: 88.31%, Time: 7.26s *
Epoch: 10, Train Loss: 0.06041, Train Acc: 98.07%, Val Loss: 0.5409, Val Acc: 88.10%, Time: 7.25s 
Epoch: 11, Train Los

Epoch: 1, Train Loss:  1.538, Train Acc: 47.63%, Val Loss:  1.275, Val Acc: 61.37%, Time: 9.18s *
Epoch: 2, Train Loss: 0.8077, Train Acc: 73.75%, Val Loss: 0.9288, Val Acc: 72.71%, Time: 7.39s *
Epoch: 3, Train Loss: 0.5235, Train Acc: 83.22%, Val Loss: 0.9671, Val Acc: 72.24%, Time: 7.41s 
Epoch: 4, Train Loss: 0.3508, Train Acc: 89.04%, Val Loss: 0.9902, Val Acc: 74.33%, Time: 7.38s *
Epoch: 5, Train Loss: 0.2513, Train Acc: 91.91%, Val Loss:  1.041, Val Acc: 75.71%, Time: 7.46s *
Epoch: 6, Train Loss: 0.1908, Train Acc: 93.99%, Val Loss: 0.9891, Val Acc: 79.49%, Time: 7.38s *
Epoch: 7, Train Loss: 0.1442, Train Acc: 95.33%, Val Loss:  1.098, Val Acc: 76.33%, Time: 7.29s 
Epoch: 8, Train Loss: 0.1121, Train Acc: 96.32%, Val Loss:  1.284, Val Acc: 75.87%, Time: 7.40s 
Epoch: 9, Train Loss: 0.0933, Train Acc: 96.74%, Val Loss:  1.223, Val Acc: 75.71%, Time: 7.20s 
Epoch: 10, Train Loss: 0.07921, Train Acc: 97.18%, Val Loss:  1.295, Val Acc: 76.72%, Time: 7.32s 
No optimization for a l

...

Epoch: 11, Train Loss: 0.1214, Train Acc: 96.08%, Val Loss:  1.804, Val Acc: 66.62%, Time: 7.32s *
Epoch: 12, Train Loss: 0.09913, Train Acc: 96.62%, Val Loss:  1.909, Val Acc: 67.19%, Time: 7.38s *
Epoch: 13, Train Loss: 0.07329, Train Acc: 97.56%, Val Loss:  1.994, Val Acc: 67.54%, Time: 7.25s *
Epoch: 14, Train Loss: 0.06891, Train Acc: 97.56%, Val Loss:  2.159, Val Acc: 66.83%, Time: 7.23s 
Epoch: 15, Train Loss: 0.05308, Train Acc: 98.26%, Val Loss:  1.957, Val Acc: 68.11%, Time: 7.24s *
Epoch: 16, Train Loss: 0.04048, Train Acc: 98.71%, Val Loss:  2.386, Val Acc: 64.63%, Time: 7.24s 
Epoch: 17, Train Loss: 0.04136, Train Acc: 98.56%, Val Loss:  2.303, Val Acc: 66.69%, Time: 7.29s 
Epoch: 18, Train Loss: 0.04433, Train Acc: 98.50%, Val Loss:  2.459, Val Acc: 65.20%, Time: 7.15s 
Epoch: 19, Train Loss: 0.05726, Train Acc: 98.05%, Val Loss:   2.14, Val Acc: 68.04%, Time: 7.38s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                      

Epoch: 6, Train Loss: 0.4464, Train Acc: 85.68%, Val Loss:  1.044, Val Acc: 71.20%, Time: 7.22s 
Epoch: 7, Train Loss: 0.3375, Train Acc: 89.32%, Val Loss:  1.212, Val Acc: 69.99%, Time: 7.18s 
Epoch: 8, Train Loss: 0.2581, Train Acc: 91.87%, Val Loss:  1.304, Val Acc: 70.71%, Time: 7.20s 
Epoch: 9, Train Loss: 0.1983, Train Acc: 93.84%, Val Loss:    1.4, Val Acc: 72.13%, Time: 7.43s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.55      0.72      0.62       148
                   Filter       0.30      0.59      0.40        87
    Compute Derived Value       0.63      0.69      0.66       192
            Find Extremum       0.79      0.81      0.80       216
                     Sort       0.86      0.88      0.87       112
          Determine Range       0.73      0.38      0.50       121
Characterize Distribution       0.96      0.75      0.85  

Epoch: 11, Train Loss: 0.07834, Train Acc: 97.43%, Val Loss:  2.236, Val Acc: 59.82%, Time: 7.59s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.56      0.76      0.64        87
                   Filter       0.20      0.45      0.27        60
    Compute Derived Value       0.89      0.37      0.52       159
            Find Extremum       0.84      0.71      0.77       119
                     Sort       0.83      0.82      0.82        60
          Determine Range       0.62      0.58      0.60       104
Characterize Distribution       0.60      0.71      0.65        73
           Find Anomalies       0.47      0.71      0.56        62
                  Cluster       0.75      0.77      0.76        82
                Correlate       0.76      0.47      0.58        95

                micro avg       0.61      0.61      0.61       901
           

Epoch: 5, Train Loss: 0.2845, Train Acc: 91.12%, Val Loss:  1.285, Val Acc: 67.65%, Time: 7.42s *
Epoch: 6, Train Loss: 0.2175, Train Acc: 93.20%, Val Loss:   1.44, Val Acc: 65.71%, Time: 7.42s 
Epoch: 7, Train Loss:  0.162, Train Acc: 94.84%, Val Loss:  1.521, Val Acc: 67.10%, Time: 7.55s 
Epoch: 8, Train Loss: 0.1318, Train Acc: 95.70%, Val Loss:   1.81, Val Acc: 62.38%, Time: 7.34s 
Epoch: 9, Train Loss: 0.1039, Train Acc: 96.60%, Val Loss:  1.954, Val Acc: 61.09%, Time: 7.33s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.29      0.21      0.24       108
                   Filter       0.24      0.57      0.34        86
    Compute Derived Value       0.58      0.47      0.52       102
            Find Extremum       0.98      0.37      0.54       143
                     Sort       0.97      0.89      0.93       127
          Determine Range  

Epoch: 1, Train Loss:  1.651, Train Acc: 43.69%, Val Loss:  1.957, Val Acc: 47.28%, Time: 9.16s *
Epoch: 2, Train Loss:  0.876, Train Acc: 71.45%, Val Loss:  2.177, Val Acc: 52.02%, Time: 7.20s *
Epoch: 3, Train Loss: 0.5685, Train Acc: 82.21%, Val Loss:  2.147, Val Acc: 54.94%, Time: 7.17s *
Epoch: 4, Train Loss: 0.3949, Train Acc: 87.85%, Val Loss:  2.315, Val Acc: 53.76%, Time: 7.10s 
Epoch: 5, Train Loss: 0.2716, Train Acc: 91.53%, Val Loss:  2.535, Val Acc: 54.25%, Time: 7.39s 
Epoch: 6, Train Loss: 0.2016, Train Acc: 93.66%, Val Loss:  2.452, Val Acc: 58.08%, Time: 7.12s *
Epoch: 7, Train Loss: 0.1514, Train Acc: 95.14%, Val Loss:  2.598, Val Acc: 57.17%, Time: 7.04s 
Epoch: 8, Train Loss:  0.121, Train Acc: 96.07%, Val Loss:  2.565, Val Acc: 59.12%, Time: 7.19s *
Epoch: 9, Train Loss: 0.09044, Train Acc: 97.01%, Val Loss:  2.595, Val Acc: 57.45%, Time: 7.09s 
Epoch: 10, Train Loss: 0.07386, Train Acc: 97.64%, Val Loss:  3.051, Val Acc: 55.57%, Time: 7.22s 
Epoch: 11, Train Loss: